# ResNet50 & VGG16 Pre-Trained Models

In [ ]:
# Download data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [ ]:
# Unzip
!unzip concrete_data_week4.zip

In [1]:
from keras.preprocessing.image import ImageDataGenerator

import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.preprocessing import image

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


In [2]:
batch_size_training = 10
batch_size_validation = 10

In [3]:
data_generator = ImageDataGenerator(preprocessing_function = preprocess_input,)

In [4]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size = (224, 224),
    batch_size = batch_size_training,
    class_mode = 'categorical')

Found 30001 images belonging to 2 classes.


In [5]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size = (224, 224),
    batch_size = batch_size_validation,
    class_mode = 'categorical')

Found 9501 images belonging to 2 classes.


In [7]:
model = Sequential()

model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',))

model.add(Dense(2, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
model.layers[0].trainable = False

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
steps_per_epoch_training = 50
steps_per_epoch_validation = 50

num_epochs = 2

In [12]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch = steps_per_epoch_training,
    epochs = num_epochs,
    validation_data = validation_generator,
    validation_steps = steps_per_epoch_validation,
    verbose = 1,)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
50/50 [==============================] - 546s 11s/step - loss: 0.5081 - acc: 0.7280 - val_loss: 0.1756 - val_acc: 0.9640
Epoch 2/2
50/50 [==============================] - 743s 15s/step - loss: 0.1289 - acc: 0.9820 - val_loss: 0.1040 - val_acc: 0.9760


In [13]:
model.save('Classifier_VGG16_model.h5')

In [6]:
from keras.models import load_model

In [7]:
resnet_m = load_model('Classifier_resnet_model.h5')
vgg16_m = load_model('Classifier_VGG16_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/opt/conda/envs/Python36/lib/python3.6/site-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [8]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size = (224, 224),
    shuffle = False)

Found 500 images belonging to 2 classes.


In [9]:
scores_vgg16 = vgg16_m.evaluate_generator(test_generator,
                                          use_multiprocessing=False,
                                          steps = 10)      

In [10]:
print('VGG16 Model Accuracy: {0:0.4F}, Loss: {0:0.4F}'.format(scores_vgg16[1], scores_vgg16[0]))

VGG16 Model Accuracy: 0.9875, Loss: 0.9875


In [ ]:
scores_resnet = resnet_m.evaluate_generator(test_generator,
                                           use_multiprocessing=False,
                                           steps = 10)

In [ ]:
print('ResNet50 Model Accuracy: {0:0.4F}, Loss: {0:0.4F}'.format(scores_resnet[1], scores_resnet[0]))

In [9]:
def printres(yhat):
  for n in range(5):
    if yhat[n][0] > 0.5:
      print('Negative')
    else:
      print('Positive')

In [10]:
print('Predictions with VGG16 Model:')
printres(vgg16_m.predict_generator(test_generator, steps = 1))

Predictions with VGG16 Model:
Negative
Negative
Negative
Negative
Negative


In [ ]:
print('Predictions with ResNet50 Model:')
printres(resnet_m.predict_generator(test_generator, steps = 1))

Predictions with ResNet50 Model:
